
# Modifying Jupyter Notebook on save
<author>Dan Rerko</author> 
<code>tags: jupyter-notebook, python, javascript, vuejs</code>

_This custom config file will export Jupyter Notebooks `.ipynb` files as blog posts!_

It can be used from the command line like so:

<code>jupyter notebook --config="./scripts/jupyter_notebook_config.py"</code>

Whenever you need to write a new blog post it will autosave and HTML file to the sites
directory.

```Python
#This script helps make the site work!
# Extra configurations file for jupyter-notebook.
import io
import os
from notebook.utils import to_api_path
from bs4 import BeautifulSoup

_html_exporter = None


def deploy_post(jupyter_html):
    """Parses jupyter notebook html file into readable HTML container for blog post deployment.
    """
    post = BeautifulSoup(jupyter_html, 'lxml')

    for ele in post.find_all("div", class_="prompt input_prompt"):
        ele.decompose()

    for ele in post.find_all("div", class_="prompt output_prompt"):
        ele.decompose()

    return str(post)


def html_post_save(model, os_path, contents_manager, **kwargs):
    """Converts notebooks to HTML after save with nbconvert's HTMLExporter
    """
    from nbconvert.exporters.html import HTMLExporter

    if model['type'] != 'notebook':
        return

    global _html_exporter

    if _html_exporter is None:
        _html_exporter = HTMLExporter(parent=contents_manager)

    log = contents_manager.log

    base, ext = os.path.splitext(os_path)
    html, resources = _html_exporter.from_filename(os_path)
    html_fname = base + resources.get('output_extension', '.html')
    log.info("Saving HTML blog post /%s",
             to_api_path(html_fname, contents_manager.root_dir))

    with io.open(html_fname, 'w', encoding='utf-8') as f:
        # Parse html file above using BeautifulSoup4
        parsed_html = deploy_post(html)
        f.write(parsed_html)


c.FileContentsManager.post_save_hook = html_post_save
```